### Import libraries

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

### Load and process data

In [ ]:
if UNZIP_DATASET:
    !unzip 'training_dataset.zip'

In [ ]:
# here we import the dataset we need depending on the model. In this case train6 was created in order to contain sequences of length 206. We used train6 for the best model.

train_data = pd.DataFrame(np.load("train6.npy"))
categories = pd.DataFrame(np.load("cat6.npy",allow_pickle=True),columns=['Category']) #contains category of the series: {'A', 'B', 'C', 'D', 'E', 'F'}.

In [ ]:
window=200
telescope=6 # it depends by the dataset that we import

train_indices = list(range(7000)) + list(range(7061, len(train_data)))

X_train = train_data.iloc[train_indices, :-telescope]
Y_train = train_data.iloc[train_indices, -telescope:]
X_test = train_data.iloc[7001:7061, :-telescope]
Y_test = train_data.iloc[7001:7061, -telescope:]

# Reset of the indexes
X_train = X_train.reset_index(drop=True)
Y_train = Y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
Y_test = Y_test.reset_index(drop=True)

In [ ]:
input_shape = X_train.shape[1]
output_shape = Y_train.shape[1]
input_shape, output_shape

### Best Model

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):

    # Define the input layer
    input_layer = tfkl.Input(shape=(input_shape), name='input_layer')
    # Reshape the input in order to use bidirectional LSTM
    reshaped_layer = tfkl.Reshape((1, input_shape), name='reshape_layer')(input_layer)

    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='lstm'), name='bidirectional_lstm')(reshaped_layer)
    x = tfkl.Bidirectional(tfkl.LSTM(64))(x)

    x = tfkl.Dense(128, activation='relu', name='dense1')(x)
    x = tfkl.Dense(64, activation='relu', name='dense2')(x)

    output_layer = tfkl.Dense(output_shape, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
batch_size = 64
epochs = 200

history = model.fit(
    x = X_train,
    y = Y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=6, factor=0.1, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('DirectForecasting0_variation_phase2_206')
#del model
#model = tfk.models.load_model('DirectForecasting3')

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_test, model.predict(X_test))
print(f'Mean Squared Error: {mse}')

### Model with CONV1D

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):

    input_layer = tfkl.Input(shape=(input_shape), name='input_layer')
    reshaped_layer = tfkl.Reshape((1, input_shape), name='reshape_layer')(input_layer)

    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='lstm'), name='bidirectional_lstm')(reshaped_layer)
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv1')(x)
    x = tfkl.Conv1D(64, 3, padding='same', activation='relu', name='conv2')(x)
    x = tfkl.Bidirectional(tfkl.LSTM(32))(x)
    x = tfkl.Dense(32, activation='relu', name='dense1')(x)
    output_layer = tfkl.Dense(output_shape, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
batch_size = 64
epochs = 200

history = model.fit(
    x = X_train,
    y = Y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=6, factor=0.1, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('DirectForecasting0')
#del model
#model = tfk.models.load_model('DirectForecasting3')

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_test, model.predict(X_test))
print(f'Mean Squared Error: {mse}')

### Model with ATTENTION

In [ ]:
class attention(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = tf.keras.backend.squeeze(e, axis=-1)
        # Compute the weights
        alpha = tf.keras.backend.softmax(e)
        # Reshape to tensorFlow format
        alpha = tf.keras.backend.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = tf.keras.backend.sum(context, axis=1)
        return context

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):

    input_layer = tfkl.Input(shape=(input_shape), name='input_layer')
    reshaped_layer = tfkl.Reshape((1, input_shape), name='reshape_layer')(input_layer)

    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='lstm'), name='bidirectional_lstm')(reshaped_layer)
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv1')(x)
    x = tfkl.Conv1D(64, 3, padding='same', activation='relu', name='conv2')(x)
    x = tfkl.Conv1D(9, 3, padding='same', activation='relu', name='conv3')(x)
    output_layer = attention()(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
batch_size = 64
epochs = 200

history = model.fit(
    x = X_train,
    y = Y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=6, factor=0.1, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('DirectForecasting0_attention')
#del model
#model = tfk.models.load_model('DirectForecasting3')

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_test, model.predict(X_test))
print(f'Mean Squared Error: {mse}')